# Monad & Applicative Operators

In [105]:
(* 
   val bind : 'a t -> ('a -> b' t) -> 'b t
   val map : ('a -> 'b) -> 'b t -> 'b t
   val product : 'a t -> 'b t -> ('a * 'b) t
   val apply : ('a -> 'b) t -> 'a t -> 'b t   
*)

let return = Option.some ;;
let (let*) = Option.bind ;;
let (let+) x f = map f x ;;


let map f o = let* o = o in return (f o) ;;

let product o1 o2 = let* o1 = o1 in let* o2 = o2 in return (o1, o2) ;;
let (and+) = product

let apply f o = let* f = f in map f o ;;
let (<~>) = apply ;;

let apply_via_map_product f o = map (fun (f,o) -> f o) @@ product f o  ;;

let plus_one = fun x -> x + 1 ;;
apply (return plus_one) (Some 10) ;;

apply (apply (return (fun x y -> (x + 1) + (y + 1))) (Some 10) ) (Some 10) ;;
(return (fun x y -> (x + 1) + (y + 1))) <~> (Some 10) <~> (Some 10) ;;

val return : 'a -> 'a option = <fun>


val ( let* ) : 'a option -> ('a -> 'b option) -> 'b option = <fun>


val ( let+ ) : 'a option -> ('a -> 'b) -> 'b option = <fun>


val map : ('a -> 'b) -> 'a option -> 'b option = <fun>


val product : 'a option -> 'b option -> ('a * 'b) option = <fun>


...

val apply_via_map_product : ('a -> 'b) option -> 'a option -> 'b option =
  <fun>


val plus_one : int -> int = <fun>


- : int option = Some 11


- : int option = Some 22


- : int option = Some 22


In [27]:
module type PARSER = sig
    type 'a t
    
    val return : 'a -> 'a t
    
    val bind : 'a t -> ('a -> 'b t) -> 'b t

    val map : ('a -> 'b) -> 'a t -> 'b t
    
    val both : 'a t -> 'b t -> ('a * 'b) t
    
    val apply : ('a -> 'b) t -> 'a t -> 'b t
    
    val skip_left : 'a t -> 'b t -> 'b t
    
    val skip_right : 'a t -> 'b t -> 'a t
    
    val either : 'a t -> 'a t -> 'a t
    
    val choice : 'a t list -> 'a t
    
    val sequence : 'a t list -> 'a list t
    
    val many : 'a t -> 'a t list
    
    val many1 : 'a t -> 'a t list
    
    val lift2 : ('a -> 'b -> 'c) -> 'a t -> 'b t -> 'c t
    
    val lift3 : ('a -> 'b -> 'c -> 'd) -> 'a t -> 'b t -> 'c t -> 'd t
    
    val opt : 'a t -> 'a option t
    
    val sep : 'a t -> 'b t -> 'a list t
    
    val sep1 : 'a t -> 'b t -> 'a list t
end ;;

module type MA =
  sig
    type 'a t
    val return : 'a -> 'a t
    val bind : 'a t -> ('a -> 'b t) -> 'b t
    val map : ('a -> 'b) -> 'a t -> 'b t
    val both : 'a t -> 'b t -> ('a * 'b) t
    val apply : ('a -> 'b) t -> 'a t -> 'b t
    val skip_left : 'a t -> 'b t -> 'b t
    val skip_right : 'a t -> 'b t -> 'a t
    val either : 'a t -> 'a t -> 'a t
    val choice : 'a t list -> 'a t
    val sequence : 'a t list -> 'a list t
    val many : 'a t -> 'a t list
    val many1 : 'a t -> 'a t list
    val lift2 : ('a -> 'b -> 'c) -> 'a t -> 'b t -> 'c t
    val lift3 : ('a -> 'b -> 'c -> 'd) -> 'a t -> 'b t -> 'c t -> 'd t
    val opt : 'a t -> 'a option t
    val sep : 'a t -> 'b t -> 'a list t
    val sep1 : 'a t -> 'b t -> 'a list t
  end


In [84]:
module Parser  = struct
    type 'a parser_result = Success of 'a * string | Failure of string
    type 'a t = string -> 'a parser_result 
    let return (a: 'a) : 'a t = let q s = Success(a, s) in q
    let bind (p: 'a t) (f: 'a -> 'b t) : 'b t = let q s = match p s with Success (a, r) -> (f a) s | Failure _ as f -> f in q 
    let (let*) = bind
    let map (f: 'a -> 'b) (p:'a t) : 'b t = let* a = p in return (f a)
    let (let+) x f = map f x
    let (|>>) x f = map f x   
    let both (p1 : 'a t) (p2 : 'b t) : ('a * 'b) t = let* a = p1 in let* b = p2 in return (a, b)
    let (and+) = both
    let (<**>) = both    
    let apply (f: ('a -> 'b) t) (p : 'a t) : 'b t = let* f' = f in let* a = p in return (f' a)
    let (<~>) = apply
    let ( *> ) p1 p2 = p1 <**> p2 |>> (fun (a,_) -> a)
    let ( <* ) p1 p2 = p1 <**> p2 |>> (fun (_,b) -> b)
    let either (p1: 'a t) (p2: 'a t) : 'a t = let q s = match p1 s with Failure _ -> p2 s | Success _ as y -> y in q   
    let ( <|> ) = either
    
end ;;

module Parser :
  sig
    type 'a parser_result = Success of 'a * string | Failure of string
    type 'a t = string -> 'a parser_result
    val return : 'a -> 'a t
    val bind : 'a t -> ('a -> 'b t) -> 'b t
    val ( let* ) : 'a t -> ('a -> 'b t) -> 'b t
    val map : ('a -> 'b) -> 'a t -> 'b t
    val ( let+ ) : 'a t -> ('a -> 'b) -> 'b t
    val ( |>> ) : 'a t -> ('a -> 'b) -> 'b t
    val both : 'a t -> 'b t -> ('a * 'b) t
    val ( and+ ) : 'a t -> 'b t -> ('a * 'b) t
    val ( <**> ) : 'a t -> 'b t -> ('a * 'b) t
    val apply : ('a -> 'b) t -> 'a t -> 'b t
    val ( <~> ) : ('a -> 'b) t -> 'a t -> 'b t
    val ( *> ) : 'a t -> 'b t -> 'a t
    val ( <* ) : 'a t -> 'b t -> 'b t
    val either : 'a t -> 'a t -> 'a t
  end
